In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
import xgboost as xgb
from scipy.stats import gmean

/data/hieunm/anaconda3/envs/action-env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_df = pd.read_csv('./n_train.csv')
test_df = pd.read_csv('./n_test.csv')

In [4]:
target=["contest-tmp2m-14d__tmp2m"]

In [5]:
train = train_df.drop(['startdate', target[0]], axis=1)

In [6]:
test = test_df.drop(['startdate', 'index'], axis=1)

In [7]:
train['source'] = 'train'
test['source']  = 'test'

In [8]:
df = pd.concat([train,test],0,ignore_index=True)

/data/hieunm/anaconda3/envs/action-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  """Entry point for launching an IPython kernel.


In [9]:
prefix_cols = [
    'nmme0-tmp2m-34w',
    'nmme-tmp2m-56w',
    'nmme-tmp2m-34w',
    'nmme-prate-34w',
    'nmme-prate-56w',
    'nmme0-prate-34w',
    'nmme0-prate-56w',
    'wind-vwnd-250',
    'wind-vwnd-925',
    'wind-uwnd-250',
    'wind-hgt-850',
    'sst-2010' ,
    'wind-hgt-500',
    'wind-uwnd-925',
    'wind-hgt-10',
    'wind-hgt-100',
    'icec-2010'
]


In [10]:
for col in tqdm(prefix_cols):
    cols = [name for name in df.columns if col in name]
    
    df['{}_1'.format(col)] = df[cols].min(axis=1)
    df['{}_2'.format(col)] = df[cols].max(axis=1)
    df['{}_3'.format(col)] = df[cols].mean(axis=1)
    df['{}_4'.format(col)] = df[cols].std(axis=1)
    df['{}_5'.format(col)] = df[cols].skew(axis=1)
    
    

100%|██████████████████████████████████████████████████████████████████████████████████████| 17/17 [26:16<00:00, 92.76s/it]


In [11]:
features = [c for c in df.columns if c != 'source']

In [12]:
train_independent_corr = df[features].corr()
train_seleted_corr_columns = np.full((train_independent_corr.shape[0],), True, dtype=bool)


In [13]:
for i in range(train_independent_corr.shape[0]):
    for j in range(i + 1, train_independent_corr.shape[0]):
        if train_independent_corr.iloc[i, j] >= 0.85:           
            if train_seleted_corr_columns[j]:
                train_seleted_corr_columns[j] = False

In [14]:
train_selected_columns = df[features].columns[list(train_seleted_corr_columns)]

In [15]:
target=["contest-tmp2m-14d__tmp2m"]
X = df[df['source'] == 'train'][features].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

5

In [16]:
train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    

5it [01:04, 12.92s/it]


In [ ]:
def train_xgboost(x, y):
    print(x.shape, y.shape)
    
    xgb_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=15000, verbose=200)
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=True, early_stopping_rounds=100)
        xgb_models.append(clf)
    
    return xgb_models
xgb_models = train_xgboost(X, y)

(375734, 345) (375734,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.023907
0:	learn: 9.6655496	test: 9.6440331	best: 9.6440331 (0)	total: 90.2ms	remaining: 22m 32s
1:	learn: 9.4627375	test: 9.4417362	best: 9.4417362 (1)	total: 132ms	remaining: 16m 28s
2:	learn: 9.2634471	test: 9.2428209	best: 9.2428209 (2)	total: 177ms	remaining: 14m 46s
3:	learn: 9.0719066	test: 9.0517156	best: 9.0517156 (3)	total: 217ms	remaining: 13m 31s
4:	learn: 8.8845122	test: 8.8645347	best: 8.8645347 (4)	total: 262ms	remaining: 13m 6s
5:	learn: 8.7010778	test: 8.6819217	best: 8.6819217 (5)	total: 304ms	remaining: 12m 38s
6:	learn: 8.5221484	test: 8.5037234	best: 8.5037234 (6)	total: 345ms	remaining: 12m 17s
7:	learn: 8.3473076	test: 8.3295389	best: 8.3295389 (7)	total: 388ms	remaining: 12m 8s
8:	learn: 8.1769628	test: 8.1594402	best: 8.1594402 (8)	total: 429ms	remaining: 11m 53s
9:	learn: 8.0105082	test: 7.9934853	best: 7.9934853 (9)	total: 470ms	remaining: 11m 44s
10:	learn: 7.8486110	test: 7.8318462	best: 7.8318462 (10)	total: 514ms	remaining: 11m 40s
1

In [19]:
preds = []
for xgb_model in tqdm(xgb_models):
    preds.append(xgb_model.predict(df[df['source'] == 'test'][features].values))
s = 0
for pred in preds:
    s += pred
s = s/5
df_ex = pd.read_csv('./sample_solution.csv')
df_ex['contest-tmp2m-14d__tmp2m'] = s
df_ex.to_csv('submission_feature_engineering.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:10<00:00,  2.19s/it]

cos_sim with best submission: 0.997750438355868


In [21]:
df.columns

Index(['year', 'quarter', 'month', 'week', 'dayofyear', 'season',
       'day_of_year_sin', 'day_of_year_cos', 'week_sin', 'week_cos',
       ...
       'wind-hgt-100_1', 'wind-hgt-100_2', 'wind-hgt-100_3', 'wind-hgt-100_4',
       'wind-hgt-100_5', 'icec-2010_1', 'icec-2010_2', 'icec-2010_3',
       'icec-2010_4', 'icec-2010_5'],
      dtype='object', length=346)